### Consensus meta-analysis



In [2]:
import os
import nipype.interfaces.fsl as fsl
import nibabel
import nilearn.plotting
import matplotlib.pyplot as plt
import pandas 
%matplotlib inline

orig_dir = '/Users/poldrack/data_unsynced/NARPS/maps/orig'
output_dir = '/Users/poldrack/data_unsynced/NARPS/maps'
template_img = os.path.join(output_dir,'templates/MNI152_T1_2mm.nii.gz')
mask_img = os.path.join(output_dir,'templates/MNI152_T1_2mm_brain_mask.nii.gz')
figure_dir = os.path.join(output_dir,'figures')




/Users/poldrack/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
for hyp in [1,2,5,6,7,8,9]:
    rand = fsl.Randomise(in_file=os.path.join(output_dir,'unthresh_concat/hypo%d.nii.gz'%hyp), 
                         mask = mask_img,
                        base_name=os.path.join(output_dir,'unthresh_randomise/hypo%d'%hyp),
                        num_perm=2500,c_thresh=3,
                        one_sample_group_mean=True,tfce=True,var_smooth=True)
    print(rand.cmdline)
    rand.run()

In [ ]:


fig, ax = plt.subplots(7,1,figsize=(12,24))
thresh=0.95
hypnums=[1,2,5,6,7,8,9]
cut_coords = [-24,-10,4,18,32,52,64]
hypotheses= {1:'+gain: equal indiff',
            2:'+gain: equal range',
            3:'+gain: equal indiff',
            4:'+gain: equal range',
            5:'-loss: equal indiff',
            6:'-loss: equal range',
            7:'+loss: equal indiff',
            8:'+loss: equal range',
            9:'+loss:ER>EI'}

for i,hyp in enumerate(hypnums):
    pmap = os.path.join(output_dir,'unthresh_randomise/hypo%d_clustere_corrp_tstat1.nii.gz'%hyp)
    tmap =  os.path.join(output_dir,'unthresh_randomise/hypo%d_tstat1.nii.gz'%hyp)
    pimg = nibabel.load(pmap)
    timg = nibabel.load(tmap)
    threshdata = (pimg.get_data()>thresh)*timg.get_data()
    threshimg = nibabel.Nifti1Image(threshdata,affine=timg.affine)
    nilearn.plotting.plot_stat_map(threshimg, threshold=0.1, display_mode="z", 
                colorbar=True,title='hyp %d:'%hyp+hypotheses[hyp],vmax=8,cmap='jet',
                                  cut_coords = cut_coords,axes = ax[i])

plt.savefig(os.path.join(figure_dir,'consensus_map.pdf'))


### run models with regressors for analysis features

Include the following regressors:
- was fmriprep used?
- was motion regression used?
- was confound regression (compcor or other) used?
- was a separate RT regressor used?





In [86]:
# create design files

metadata_file = '/Users/poldrack/data_unsynced/NARPS/analysis_pipelines_SW.xlsx'
metadata = pandas.read_excel(metadata_file,header=1)
metadata['collectionID']=None
for i in metadata.index:
    metadata.loc[i,'collectionID']= '%s_%s'%(metadata.loc[i,'NV_collection_string'].strip(),metadata.loc[i,'teamID'].strip())
metadata.index = metadata.collectionID
metadata['used_fmriprep_data'] = (metadata['used_fmriprep_data']=='Yes').astype('int')*2 - 1
metadata.shape


(70, 42)

In [93]:
columns_to_use = ['used_fmriprep_data']
md = metadata[columns_to_use]

analysis_dir = os.path.join(output_dir,'unthresh_randomise_fmriprep')

# just need to do it for one hypothesis since it's same for all
hyp=1
in_file=os.path.join(output_dir,'unthresh_concat/hypo%d.nii.gz'%hyp)
label_file = os.path.join(output_dir,'unthresh_concat/labels_hypo%d.txt'%hyp)
labels = pandas.read_csv(label_file,header=None)
labels.columns = ['NV_collection_string']
labels.index = labels.NV_collection_string
labels['constant']=1
nlabels=labels.shape[0]
labels = labels.merge(md,left_index=True,right_index=True)
assert labels.shape[0] == nlabels  # make sure we didn't lose any!
del labels['NV_collection_string']
 

In [96]:
# write fsl design file
designfile = os.path.join(analysis_dir,'design.txt')
with open(designfile,'w') as f:
    f.write('/NumWaves %d\n'%labels.shape[1])
    f.write('/NumPoints %d\n/Matrix\n'%labels.shape[0])
    f.write(labels.to_string(header=False,index=False))

confile = os.path.join(analysis_dir,'design.con')
with open(confile,'w') as f:
    f.write('/NumWaves 2\n')
    f.write('/NumPoints 4\n/Matrix\n')
    f.write('1\t0\n')
    f.write('-1\t0\n')
    f.write('0\t1\n')
    f.write('0\t-1\n')


In [98]:
for hyp in [5]:
    rand = fsl.Randomise(in_file=os.path.join(output_dir,'unthresh_concat/hypo%d.nii.gz'%hyp), 
                         mask = mask_img,
                        base_name=os.path.join(output_dir,'unthresh_randomise_fmriprep/hypo%d'%hyp),
                        num_perm=2500,c_thresh=3,
                        var_smooth=True,design_mat=designfile,
                        tcon=confile)
    print(rand.cmdline)
    rand.run()

randomise -i /Users/poldrack/data_unsynced/NARPS/maps/unthresh_concat/hypo5.nii.gz -o "/Users/poldrack/data_unsynced/NARPS/maps/unthresh_randomise_fmriprep/hypo5" -d /Users/poldrack/data_unsynced/NARPS/maps/unthresh_randomise_fmriprep/design.txt -t /Users/poldrack/data_unsynced/NARPS/maps/unthresh_randomise_fmriprep/design.con -c 3.0 -m /Users/poldrack/data_unsynced/NARPS/maps/templates/MNI152_T1_2mm_brain_mask.nii.gz -n 2500 -v 1
190606-17:11:31,542 nipype.interface INFO:
	 stdout 2019-06-06T17:11:31.542513:randomise options: -i /Users/poldrack/data_unsynced/NARPS/maps/unthresh_concat/hypo5.nii.gz -o /Users/poldrack/data_unsynced/NARPS/maps/unthresh_randomise_fmriprep/hypo5 -d /Users/poldrack/data_unsynced/NARPS/maps/unthresh_randomise_fmriprep/design.txt -t /Users/poldrack/data_unsynced/NARPS/maps/unthresh_randomise_fmriprep/design.con -c 3.0 -m /Users/poldrack/data_unsynced/NARPS/maps/templates/MNI152_T1_2mm_brain_mask.nii.gz -n 2500 -v 1 
190606-17:11:34,264 nipype.interface INFO:


In [63]:
cope_desc = {1:'pos',2:'neg',3:''}
for cope in range(1,5):
    for i,hyp in enumerate(hypnums):
        pmap = os.path.join(output_dir,'unthresh_randomise_fmriprep/hypo%d_clustere_corrp_tstat%d.nii.gz'%(hyp,cope))
        tmap =  os.path.join(output_dir,'unthresh_randomise_fmriprep/hypo%d_tstat%d.nii.gz'%(hyp,cope))
        pimg = nibabel.load(pmap)
        timg = nibabel.load(tmap)
        threshdata = (pimg.get_data()>thresh)*timg.get_data()
        threshimg = nibabel.Nifti1Image(threshdata,affine=timg.affine)
        nilearn.plotting.plot_stat_map(threshimg, threshold=0.1, display_mode="z", 
                    colorbar=True,title='hyp %d:'%hyp+hypotheses[hyp],vmax=8,cmap='jet',
                                      cut_coords = cut_coords,axes = ax[i])

    plt.savefig(os.path.join(figure_dir,'consensus_map_cope%d.pdf'%cope))


Index(['constant', 'used_fmriprep_data'], dtype='object')

In [71]:
labels.shape

(50, 2)